## وارد کردن کتابخانه‌های مورد نیاز  
کتابخانه‌های PyTorch و transformers را که برای بارگذاری و استفاده از مدل Phi-4 لازم هستند، وارد کنید.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## تنظیم بذر تصادفی  
بذر تصادفی را تنظیم کنید تا نتایج قابل تکرار در اجراهای مختلف تضمین شود.  


In [ ]:
torch.random.manual_seed(0)

## بارگذاری مدل و توکنایزر Phi-4-mini-flash-reasoning
مدل Microsoft Phi-4-mini-flash-reasoning و توکنایزر مربوط به آن را از Hugging Face بارگذاری کنید. این مدل برای افزایش سرعت استنتاج روی CUDA بارگذاری خواهد شد.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## آماده‌سازی پیام ورودی  
یک پیام مکالمه‌ای با یک مسئله ریاضی معادله درجه دوم ایجاد کنید و آن را با قالب چت برای مدل قالب‌بندی کنید.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## تولید پاسخ  
ایجاد یک پاسخ از مدل با استفاده از پارامترهای مشخص شده مانند دما و top_p برای کنترل تصادفی بودن خروجی.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## تبدیل خروجی به متن
تبدیل رشته‌های توکن تولید شده به متن قابل خواندن برای انسان، بدون نمایش توکن‌های ورودی اصلی، به‌طوری که فقط پاسخ مدل نمایش داده شود.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
